In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types as T
import os

In [2]:
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [3]:
def find_csv_files(folder_path):
    csv_files = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.csv'):
                csv_files.append(os.path.join(root, file))
    return csv_files

In [4]:
def show(df, limit=10):
    return df.toPandas().head(limit)

In [5]:
folder = r"C:\Development\ultimateNakMuay\data\processed"
files = find_csv_files(folder)

In [6]:
files = files[2:]

In [7]:
organizations = ["ufc", "onefc", "bellator", "glory"]

organizations = {
    k:{
        "event": spark.read.csv(list(filter(lambda x: "event" in x and k in x, files))[0], header=True),
        "result": spark.read.csv(list(filter(lambda x: "result" in x and k in x, files))[0], header=True),
        }
    for k in organizations
}

In [8]:
def get_data(org):
    return organizations[org]["event"], organizations[org]["result"]

In [9]:
event, result = get_data("ufc")

In [22]:
ufc = event.join(result, on="event_id", how="left").drop("event_id", "event_name")

In [24]:
show(ufc.orderBy(F.col("date").desc()))

,event_num,event,date,venue,city,state,country,attendance,weight_class,winner,loser,method,round,time,fight_card
0,future,UFC PPV,2023-12-16,T-Mobile Arena,Las Vegas,Nevada,US,None,None,None,None,None,None,None,None
1,future,UFC Fight Night,2023-11-18,UFC Apex,Las Vegas,Nevada,US,None,None,None,None,None,None,None,None
2,future,UFC Fight Night,2023-11-04,TBD,None,São Paulo,Brazil,None,None,None,None,None,None,None,None
3,future,UFC 294,2023-10-21,Etihad Arena,None,Abu Dhabi,United Arab Emirates,None,None,None,None,None,None,None,None
4,future,UFC Fight Night,2023-10-07,UFC Apex,Las Vegas,Nevada,US,None,None,None,None,None,None,None,None
5,future,UFC Fight Night,2023-09-23,UFC Apex,Las Vegas,Nevada,US,None,None,None,None,None,None,None,None
6,future,UFC Fight Night,2023-09-16,T-Mobile Arena,Las Vegas,Nevada,US,None,None,None,None,None,None,None,None
7,future,UFC 293,2023-09-10,Qudos Bank Arena,None,Sydney,Australia,None,None,None,None,None,None,None,None
8,future,UFC on ESPN: Gane vs. Spivak,2023-09-02,Accor Arena,None,Paris,France,None,None,None,None,None,None,None,None
9,future,UFC on ESPN: Holloway vs. The Korean Zombie,2023-08-26,Singapore Indoor Stadium,None,Kallang,Singapore,None,None,None,None,None,None,None,None
